In [1]:
from transformers import pipeline, AutoProcessor, AutoModelForVision2Seq
import torch
from PIL import Image

# 1. Define the base model and the adapter
base_model_id = "unsloth/gemma-3n-E2B-it"
adapter_id = "gigwegbe/gemma-3n-E2B-it-finetuned-adapters"

# 2. Load the base model and processor
# You'll need to specify the model, not the FastVisionModel, for the pipeline.
# Load the model with the same configurations as your original code (e.g., 4-bit quantization)
model = AutoModelForVision2Seq.from_pretrained(
    base_model_id,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
processor = AutoProcessor.from_pretrained(base_model_id)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:2160: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

ValueError: Unrecognized configuration class <class 'transformers.models.gemma3n.configuration_gemma3n.Gemma3nConfig'> for this kind of AutoModel: AutoModelForVision2Seq.
Model type should be one of BlipConfig, Blip2Config, ChameleonConfig, GitConfig, Idefics2Config, Idefics3Config, InstructBlipConfig, InstructBlipVideoConfig, Kosmos2Config, LlavaConfig, LlavaNextConfig, LlavaNextVideoConfig, LlavaOnevisionConfig, Mistral3Config, MllamaConfig, PaliGemmaConfig, Pix2StructConfig, Qwen2_5_VLConfig, Qwen2VLConfig, VideoLlavaConfig, VipLlavaConfig, VisionEncoderDecoderConfig.

In [ ]:
# 3. Load the adapter weights (using PEFT's from_pretrained or similar)
# Hugging Face's PEFT library is the standard way to handle adapters
from peft import PeftModel

# The model variable now becomes a PeftModel instance with the adapter attached
model = PeftModel.from_pretrained(model, adapter_id)


In [ ]:
# 4. Create the multimodal pipeline
# For multimodal models like this, you can specify the task as "image-to-text"
# and pass the model and processor.
pipe = pipeline(
    "image-to-text",
    model=model,
    tokenizer=processor.tokenizer,
    image_processor=processor.image_processor,
    max_new_tokens=256,
    temperature=1.0,
    top_p=0.95,
    top_k=64,
    device=0, # or 'cuda' or -1 for cpu
)


In [ ]:
# 5. Prepare the input
image = Image.open("damage.png").convert("RGB")
instruction = (
    "You are an expert automobile inspector. Describe the visible car damages in the image, "
    "mentioning damage types and approximate regions."
)

# 6. Run the pipeline with the formatted prompt and image
# The pipeline handles the tokenization and generation for you.
output = pipe(image, instruction)

# The output is a list of dictionaries, so you can access the generated text
print(output[0]['generated_text'])
